# FASE 1:INGESTA & ORGANIZACIÓN

Este notebook documenta la **FASE 1 del proyecto** de análisis de servicios públicos domiciliarios en Barranquilla. Aquí se realiza la preparación integral de los datos para análisis posterior y visualización en Power BI.

---

## ✅ Contenido del notebook

- 📥 Carga de datos crudos desde múltiples fuentes
- 🧾 Exploración estructural: dimensiones, tipos, nulos, duplicados
- 🧹 Limpieza detallada de cada dataset:
  - Normalización de nombres de columnas
  - Conversión de tipos
  - Unificación de estructuras
  - Limpieza de texto y tildes
- 📅 Generación de variables temporales (como `Fecha`)
- 🔁 Reformateo de tablas a estructuras largas y analizables
- 💾 Exportación de archivos limpios a la carpeta `/01_data_clean/`

---

## 📂 Archivos utilizados en bruto (carpeta `00_data_raw/`)

- `Reclamaciones_Aseo.csv`
- `Reclamaciones_Energia_Electrica.csv`
- `Reclamaciones_Acueducto.csv`
- `Reclamaciones_Alcantarillado.csv`
- `Reclamaciones_Gas_Natural.csv`
- `Caracterizacion_Empresas.csv`
- `consumo_servicios_barranquilla_simulado.csv`
- `consumo_servicios_barranquilla_simulado_2.csv`



###  🔢 **1. Importar Librerías Esenciales**

In [77]:
import pandas as pd
import numpy as np
import os

# Opcional: mostrar todas las columnas al inspeccionar dataframes
pd.set_option('display.max_columns', None)


*Esto carga las librerías esenciales para procesamiento de datos y establece la visualización sin truncar columnas.*

### 📂 **2. Definir Rutas y Directorios**





In [94]:
from google.colab import drive

# Montar Google Drive (si estás usando Colab)
drive.mount('/content/drive')

# Definir paths del proyecto
base_path = '/content/drive/MyDrive/00_Data_ISES'
raw_path = os.path.join(base_path, '00_data_raw')
clean_path = os.path.join(base_path, '01_data_clean')

# Cambiar a carpeta de datos
os.chdir(raw_path)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


*Esto organiza tus datos correctamente por carpetas y prepara el entorno para leer y guardar datos.*

###📄 **3. Leer Archivos CSV Originales**


In [95]:
# ------------------------------
# LECTURA DE TODOS LOS DATOS CRUDOS
# ------------------------------

# Leer archivos de reclamaciones
df_aseo = pd.read_csv("Reclamaciones_Aseo.csv", encoding='latin1', skiprows=2)
df_energia = pd.read_csv("Reclamaciones_Energia_Electrica.csv", encoding='latin1', skiprows=2)
df_acueducto = pd.read_csv("Reclamaciones_Acueducto.csv", encoding='latin1', skiprows=2)
df_alcantarillado = pd.read_csv("Reclamaciones_Alcantarillado.csv", encoding='latin1', skiprows=2)
df_gas = pd.read_csv("Reclamaciones_Gas_Natural.csv", encoding='latin1', skiprows=2)

# Leer archivo de caracterización de empresas
df_empresas = pd.read_csv("Caracterizacion_Empresas.csv", encoding='latin1', skiprows=3)

# Eliminar la columna de índice 3 (la cuarta)
df_empresas.drop(df_empresas.columns[4], axis=1, inplace=True)

# Leer archivos de consumo simulado
df_consumo1 = pd.read_csv("consumo_servicios_barranquilla_simulado.csv", encoding='latin1')
df_consumo2 = pd.read_csv("consumo_servicios_barranquilla_simulado_2.csv", encoding='latin1')

# Unir ambos
df_consumo = pd.concat([df_consumo1, df_consumo2], ignore_index=True)

# 🔹 3. Función para corregir caracteres especiales
def limpiar_texto(texto):
    if isinstance(texto, str):
        try:
            return texto.encode('latin1').decode('utf-8')
        except:
            return texto.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
    return texto

# 🔹 4. Aplicar limpieza a columnas tipo texto
for col in df_consumo.select_dtypes(include=['object']).columns:
    df_consumo[col] = df_consumo[col].apply(limpiar_texto)



`skiprows=2 ` es para omitir las dos primeras filas y tratar la tercera como encabezado real.


***Verficacion***

In [96]:
# Verificar las primeras filas de cada archivo
for name, df in {
    "Aseo": df_aseo,
    "Energía": df_energia,
    "Acueducto": df_acueducto,
    "Alcantarillado": df_alcantarillado,
    "Gas": df_gas,
    "Empresas": df_empresas,
    "Consumo 1": df_consumo1,
    "Consumo 2": df_consumo2,
    "Consumo": df_consumo
}.items():
    print(f"\n▶️ Dataset: {name}")
    display(df.head(3))



▶️ Dataset: Aseo


,MUNICIPIO,TIPO DE TRÁMITE,DETALLE,2021,2022,2023,2024,2025
0,BARANOA,QUEJA,INTERRUPCIONES EN LA PRESTACION DEL SERVICIO,51,39,42,91,45
1,BARANOA,QUEJA,VARIACIONES EN LAS CARACTERISTICAS DEL SUMINIS...,20,18,21,25,9
2,BARANOA,QUEJA,AFECTACION AMBIENTAL,6,11,11,12,3



▶️ Dataset: Energía


,MUNICIPIO,TIPO DE TRÁMITE,DETALLE,2021,2022,2023,2024,2025
0,BARANOA,QUEJA,INTERRUPCIONES EN LA PRESTACION DEL SERVICIO,28,85,33,34,16
1,BARANOA,RECLAMACION,INCONFORMIDAD CON LA MEDICION DEL CONSUMO O PR...,"4,112","5,809","7,018","3,054",684
2,BARANOA,RECLAMACION,VARIACIONES EN LAS CARACTERISTICAS DEL SUMINIS...,"6,284","2,928","3,398","2,262",947



▶️ Dataset: Acueducto


...

,fecha,barrio,estrato,consumo_energia_kwh,consumo_agua_m3,consumo_gas_m3,reclamaciones,interrupciones,duracion_promedio_interrupciones,estado_medidor,tecnico_asignado,consumo_sospechoso,fraude_detectado,lectura_estimada
0,2023-01-01,La SierrA,1,219.737132,18.617357,19.533820,1,0,50.230299,NaN,NaN,NaN,0,no
1,2023-01-01,la sierra,5,202.323303,30.105153,10.933853,1,1,29.748302,bien,Laura PÃ©rez,NaN,0,sÃ­
2,2023-01-01,La siera,3,245.315606,4.761240,12.003678,2,0,27.575932,bien,Carlos Ruiz,NaN,0,no



▶️ Dataset: Consumo 2


,fecha,barrio,estrato,consumo_energia_kwh,consumo_agua_m3,consumo_gas_m3,reclamaciones,interrupciones,duracion_promedio_interrupciones,estado_medidor,tecnico_asignado,consumo_sospechoso,fraude_detectado,lectura_estimada
0,2022-01-01,Las Mercedes,5,0.000000,9.147541,14.045894,0,0,29.256765,bien,MarÃ­a G.,no,0,no
1,2022-01-01,Los Robles,4,140.633888,34.390844,17.792820,2,1,50.644936,mal calibrado,MarÃ­a G.,NaN,0,no
2,2022-01-01,Santa MarÃ­a,5,254.439027,24.040628,14.782587,3,1,31.011870,daÃ±ado,NaN,NaN,0,no



▶️ Dataset: Consumo


,fecha,barrio,estrato,consumo_energia_kwh,consumo_agua_m3,consumo_gas_m3,reclamaciones,interrupciones,duracion_promedio_interrupciones,estado_medidor,tecnico_asignado,consumo_sospechoso,fraude_detectado,lectura_estimada
0,2023-01-01,La SierrA,1,219.737132,18.617357,19.533820,1,0,50.230299,NaN,NaN,NaN,0,no
1,2023-01-01,la sierra,5,202.323303,30.105153,10.933853,1,1,29.748302,bien,Laura Pérez,NaN,0,sí
2,2023-01-01,La siera,3,245.315606,4.761240,12.003678,2,0,27.575932,bien,Carlos Ruiz,NaN,0,no


In [97]:
# Verificar encabezados correctos tras ajustes
print("🔎 Columnas de Aseo:", df_aseo.columns.tolist())
print("🔎 Columnas de Consumo Unificado:", df_consumo.columns.tolist())


🔎 Columnas de Aseo: ['MUNICIPIO', 'TIPO DE TRÁMITE', 'DETALLE', '2021', '2022', '2023', '2024', '2025']
🔎 Columnas de Consumo Unificado: ['fecha', 'barrio', 'estrato', 'consumo_energia_kwh', 'consumo_agua_m3', 'consumo_gas_m3', 'reclamaciones', 'interrupciones', 'duracion_promedio_interrupciones', 'estado_medidor', 'tecnico_asignado', 'consumo_sospechoso', 'fraude_detectado', 'lectura_estimada']


###🧼 **4. Exploración de Datos**

####*4.1 Función Auxiliar de Diagnóstico (para uso general)*

In [100]:
def diagnostico_df(nombre, df):
    print(f"\n📊 Análisis del dataset: {nombre}")
    print("-" * 50)
    print(f"▶️ Dimensiones: {df.shape}")
    print(f"▶️ Columnas:\n{df.columns.tolist()}")
    print("▶️ Tipos de datos:")
    print(df.dtypes)
    print("\n▶️ Valores nulos:")
    print(df.isnull().sum())
    print(f"▶️ Filas duplicadas: {df.duplicated().sum()}")
    display(df.head(3))


####*4.2 Aplicar Diagnóstico a Todos los DataFrames*


In [101]:
diagnostico_df("Aseo", df_aseo)
diagnostico_df("Energía", df_energia)
diagnostico_df("Acueducto", df_acueducto)
diagnostico_df("Alcantarillado", df_alcantarillado)
diagnostico_df("Gas", df_gas)
diagnostico_df("Empresas", df_empresas)
diagnostico_df("Consumo (unificado)", df_consumo)



📊 Análisis del dataset: Aseo
--------------------------------------------------
▶️ Dimensiones: (734, 8)
▶️ Columnas:
['MUNICIPIO', 'TIPO DE TRÁMITE', 'DETALLE', '2021', '2022', '2023', '2024', '2025']
▶️ Tipos de datos:
MUNICIPIO          object
TIPO DE TRÁMITE    object
DETALLE            object
2021               object
2022               object
2023               object
2024               object
2025               object
dtype: object

▶️ Valores nulos:
MUNICIPIO          0
TIPO DE TRÁMITE    0
DETALLE            0
2021               0
2022               0
2023               0
2024               0
2025               0
dtype: int64
▶️ Filas duplicadas: 0


,MUNICIPIO,TIPO DE TRÁMITE,DETALLE,2021,2022,2023,2024,2025
0,BARANOA,QUEJA,INTERRUPCIONES EN LA PRESTACION DEL SERVICIO,51,39,42,91,45
1,BARANOA,QUEJA,VARIACIONES EN LAS CARACTERISTICAS DEL SUMINIS...,20,18,21,25,9
2,BARANOA,QUEJA,AFECTACION AMBIENTAL,6,11,11,12,3



📊 Análisis del dataset: Energía
--------------------------------------------------
▶️ Dimensiones: (1308, 8)
▶️ Columnas:
['MUNICIPIO', 'TIPO DE TRÁMITE', 'DETALLE', '2021', '2022', '2023', '2024', '2025']
▶️ Tipos de datos:
MUNICIPIO          object
TIPO DE TRÁMITE    object
DETALLE            object
2021               object
2022               object
2023               object
2024               object
2025               object
dtype: object

▶️ Valores nulos:
MUNICIPIO          0
TIPO DE TRÁMITE    0
DETALLE            0
2021               0
2022               0
2023               0
2024               0
2025               0
dtype: int64
▶️ Filas duplicadas: 0


,MUNICIPIO,TIPO DE TRÁMITE,DETALLE,2021,2022,2023,2024,2025
0,BARANOA,QUEJA,INTERRUPCIONES EN LA PRESTACION DEL SERVICIO,28,85,33,34,16
1,BARANOA,RECLAMACION,INCONFORMIDAD CON LA MEDICION DEL CONSUMO O PR...,"4,112","5,809","7,018","3,054",684
2,BARANOA,RECLAMACION,VARIACIONES EN LAS CARACTERISTICAS DEL SUMINIS...,"6,284","2,928","3,398","2,262",947



📊 Análisis del dataset: Acueducto
--------------------------------------------------
▶️ Dimensiones: (1171, 8)
▶️ Columnas:
['MUNICIPIO', 'TIPO DE TRÁMITE', 'DETALLE', '2021', '2022', '2023', '2024', '2025']
▶️ Tipos de datos:
MUNICIPIO          object
TIPO DE TRÁMITE    object
DETALLE            object
2021               object
2022               object
2023               object
2024               object
2025               object
dtype: object

▶️ Valores nulos:
MUNICIPIO          0
TIPO DE TRÁMITE    0
DETALLE            0
2021               0
2022               0
2023               0
2024               0
2025               0
dtype: int64
▶️ Filas duplicadas: 0


...

,MUNICIPIO,TIPO DE TRÁMITE,DETALLE,2021,2022,2023,2024,2025
0,BARANOA,QUEJA,ENTREGA INOPORTUNA O NO ENTREGA DE LA FACTURA,19,35,52,19,4
1,BARANOA,QUEJA,SUSPENSION O CORTE DEL SERVICIO,10,23,24,7,5
2,BARANOA,QUEJA,NO CONEXION DEL SERVICIO,0,2,0,0,0



📊 Análisis del dataset: Empresas
--------------------------------------------------
▶️ Dimensiones: (1726, 6)
▶️ Columnas:
['MUNICIPIO', 'EMPRESA', 'Medidas', 'ACUEDUCTO', 'ALCANTARILLADO', 'ENERGIA ELECTRICA']
▶️ Tipos de datos:
MUNICIPIO            object
EMPRESA              object
Medidas              object
ACUEDUCTO            object
ALCANTARILLADO       object
ENERGIA ELECTRICA    object
dtype: object

▶️ Valores nulos:
MUNICIPIO              0
EMPRESA                0
Medidas                0
ACUEDUCTO            736
ALCANTARILLADO       755
ENERGIA ELECTRICA     91
dtype: int64
▶️ Filas duplicadas: 0


,MUNICIPIO,EMPRESA,Medidas,ACUEDUCTO,ALCANTARILLADO,ENERGIA ELECTRICA
0,BARANOA,"82-SOCIEDAD DE ACUEDUCTO, ALCANTARILLADO Y ASE...",Facturación total,"61,253,932,570","6,988,757,679",0
1,BARANOA,"82-SOCIEDAD DE ACUEDUCTO, ALCANTARILLADO Y ASE...",Consumo total,"3,850,631.0","589,215.0",0.0
2,BARANOA,"82-SOCIEDAD DE ACUEDUCTO, ALCANTARILLADO Y ASE...",Numero de suscriptores,"11,892.00","6,758.00",NaN



📊 Análisis del dataset: Consumo (unificado)
--------------------------------------------------
▶️ Dimensiones: (326, 14)
▶️ Columnas:
['fecha', 'barrio', 'estrato', 'consumo_energia_kwh', 'consumo_agua_m3', 'consumo_gas_m3', 'reclamaciones', 'interrupciones', 'duracion_promedio_interrupciones', 'estado_medidor', 'tecnico_asignado', 'consumo_sospechoso', 'fraude_detectado', 'lectura_estimada']
▶️ Tipos de datos:
fecha                                object
barrio                               object
estrato                               int64
consumo_energia_kwh                 float64
consumo_agua_m3                     float64
consumo_gas_m3                      float64
reclamaciones                         int64
interrupciones                        int64
duracion_promedio_interrupciones    float64
estado_medidor                       object
tecnico_asignado                     object
consumo_sospechoso                   object
fraude_detectado                      int64
lectura_esti

,fecha,barrio,estrato,consumo_energia_kwh,consumo_agua_m3,consumo_gas_m3,reclamaciones,interrupciones,duracion_promedio_interrupciones,estado_medidor,tecnico_asignado,consumo_sospechoso,fraude_detectado,lectura_estimada
0,2023-01-01,La SierrA,1,219.737132,18.617357,19.533820,1,0,50.230299,NaN,NaN,NaN,0,no
1,2023-01-01,la sierra,5,202.323303,30.105153,10.933853,1,1,29.748302,bien,Laura Pérez,NaN,0,sí
2,2023-01-01,La siera,3,245.315606,4.761240,12.003678,2,0,27.575932,bien,Carlos Ruiz,NaN,0,no


###🧹 **5. Limpieza de Datos por Tipo de Archivo**

####*5.1 Reclamaciones*

**Verificacion de Encabezado**

In [103]:
print(df_aseo.columns.tolist())


['MUNICIPIO', 'TIPO DE TRÁMITE', 'DETALLE', '2021', '2022', '2023', '2024', '2025']


In [104]:
def limpiar_reclamaciones_tabla_transpuesta(df):
    df = df.copy()

    # Eliminar duplicados
    df.drop_duplicates(inplace=True)

    # Rellenar valores faltantes
    df.fillna('', inplace=True)

    # Convertir de formato ancho a largo (melt)
    df_limpio = pd.melt(
        df,
        id_vars=['MUNICIPIO', 'TIPO DE TRÁMITE', 'DETALLE'],
        value_vars=['2021', '2022', '2023', '2024', '2025'],
        var_name='Año',
        value_name='Número de Reclamos'
    )

    # Convertir Año a numérico
    df_limpio['Año'] = pd.to_numeric(df_limpio['Año'], errors='coerce')

    # Crear columna de fecha (1 de enero del año)
    df_limpio['Fecha'] = pd.to_datetime(df_limpio['Año'].astype(str) + '-01-01')

    # ✅ LIMPIAR COMAS y convertir a entero
    df_limpio['Número de Reclamos'] = (
        df_limpio['Número de Reclamos']
        .astype(str)
        .str.replace(",", "", regex=False)
        .str.strip()
        .replace("nan", np.nan)
        .astype(float)
        .fillna(0)
        .astype(int)
    )

    return df_limpio



In [105]:
# Aplicar limpieza
df_aseo = limpiar_reclamaciones_tabla_transpuesta(df_aseo)
df_energia = limpiar_reclamaciones_tabla_transpuesta(df_energia)
df_acueducto = limpiar_reclamaciones_tabla_transpuesta(df_acueducto)
df_alcantarillado = limpiar_reclamaciones_tabla_transpuesta(df_alcantarillado)
df_gas = limpiar_reclamaciones_tabla_transpuesta(df_gas)


####*5.2 Consumo Simulado*

In [106]:
print(df_consumo.columns.tolist())


['fecha', 'barrio', 'estrato', 'consumo_energia_kwh', 'consumo_agua_m3', 'consumo_gas_m3', 'reclamaciones', 'interrupciones', 'duracion_promedio_interrupciones', 'estado_medidor', 'tecnico_asignado', 'consumo_sospechoso', 'fraude_detectado', 'lectura_estimada']


In [107]:
# Verificación de columnas
print(df_consumo.columns)

# Eliminar duplicados
df_consumo.drop_duplicates(inplace=True)

# Revisión de tipos y nulos
df_consumo.info()
print(df_consumo.isnull().sum())

# ---------------------------
# LIMPIEZA Y CORRECCIÓN DE BARRIOS
# ---------------------------

# Paso 1: Función para estandarizar nombre de barrio
def limpiar_barrio(valor):
    if pd.isnull(valor):
        return np.nan
    valor = str(valor)
    valor = valor.strip()
    valor = valor.upper()
    valor = valor.replace('-', ' ')
    valor = valor.replace('_', ' ')
    valor = ''.join(c for c in valor if c.isalnum() or c.isspace())
    valor = ' '.join(valor.split())
    return valor

# Paso 2: Aplicar limpieza desde la columna 'barrio'
df_consumo['BARRIO_LIMPIO'] = df_consumo['barrio'].apply(limpiar_barrio)

# Paso 3: Correcciones manuales
correcciones_barrios = {
    'SIMON BOLIVAR': 'SIMÓN BOLÍVAR',
    'JOSE ANTONIO GALAN': 'JOSÉ ANTONIO GALÁN',
    'RECREO': 'EL RECREO',
    'LAS NIEVES': 'LAS NIEVES',
    'CIUDADELA 20 DE JULIO': 'CIUDADELA 20 JULIO',
    'LA ESPERANZA': 'LA ESPERANZA',
    'SANTO DOMINGO': 'SANTO DOMINGO',
    'LA LUZ': 'LUZ',
    'El Prado':'EL PRADO',
    'LA SIERA':'LA SIERRA'

}
df_consumo['BARRIO_LIMPIO'] = df_consumo['BARRIO_LIMPIO'].replace(correcciones_barrios)

# Paso 4: Ver resultado final
print("✅ Barrios corregidos:")
display(df_consumo[['barrio', 'BARRIO_LIMPIO']].drop_duplicates().sort_values('BARRIO_LIMPIO').head(20))


Index(['fecha', 'barrio', 'estrato', 'consumo_energia_kwh', 'consumo_agua_m3',
       'consumo_gas_m3', 'reclamaciones', 'interrupciones',
       'duracion_promedio_interrupciones', 'estado_medidor',
       'tecnico_asignado', 'consumo_sospechoso', 'fraude_detectado',
       'lectura_estimada'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 306 entries, 0 to 315
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   fecha                             306 non-null    object 
 1   barrio                            306 non-null    object 
 2   estrato                           306 non-null    int64  
 3   consumo_energia_kwh               297 non-null    float64
 4   consumo_agua_m3                   293 non-null    float64
 5   consumo_gas_m3                    289 non-null    float64
 6   reclamaciones                     306 non-null    int64  
 7   interrupc

,barrio,BARRIO_LIMPIO
230,Altos del Prado,ALTOS DEL PRADO
239,Betania,BETANIA
236,Campo Alegre,CAMPO ALEGRE
9,Ciudadela 20 de Julio,CIUDADELA 20 JULIO
3,El Prado,EL PRADO
4,el prado,EL PRADO
231,El Tabor,EL TABOR
237,La Concepción,LA CONCEPCIÓN
0,La SierrA,LA SIERRA
2,La siera,LA SIERRA


####*5.3 Caracterizacion de Empresa*

In [89]:
# Paso 1: Convertir valores a numéricos eliminando comas y puntos innecesarios
servicios = ['ACUEDUCTO', 'ALCANTARILLADO', 'ENERGIA ELECTRICA']
for col in servicios:
    df_empresas[col] = (
        df_empresas[col]
        .astype(str)
        .str.replace(",", "", regex=False)
        .str.replace(" ", "")
        .replace("nan", np.nan)
        .astype(float)
    )

# Paso 2: Convertir de formato ancho a largo
df_empresas_limpio = pd.melt(
    df_empresas,
    id_vars=['MUNICIPIO', 'EMPRESA', 'Medidas'],
    value_vars=servicios,
    var_name='SERVICIO',
    value_name='VALOR'
)

# Paso 3: Eliminar filas sin valor
df_empresas_limpio = df_empresas_limpio.dropna(subset=['VALOR'])

# Paso 4: Renombrar columna 'Medidas' a 'MEDIDA' para consistencia
df_empresas_limpio.rename(columns={'Medidas': 'MEDIDA'}, inplace=True)

# Paso 5: Confirmar resultado
print("✅ Tabla EMPRESAS reformateada:")
display(df_empresas_limpio.head())



✅ Tabla EMPRESAS reformateada:


,MUNICIPIO,EMPRESA,MEDIDA,SERVICIO,VALOR
0,BARANOA,"82-SOCIEDAD DE ACUEDUCTO, ALCANTARILLADO Y ASE...",Facturación total,ACUEDUCTO,6.125393e+10
1,BARANOA,"82-SOCIEDAD DE ACUEDUCTO, ALCANTARILLADO Y ASE...",Consumo total,ACUEDUCTO,3.850631e+06
2,BARANOA,"82-SOCIEDAD DE ACUEDUCTO, ALCANTARILLADO Y ASE...",Numero de suscriptores,ACUEDUCTO,1.189200e+04
3,BARANOA,"82-SOCIEDAD DE ACUEDUCTO, ALCANTARILLADO Y ASE...",Subsidios,ACUEDUCTO,2.558589e+09
4,BARANOA,"82-SOCIEDAD DE ACUEDUCTO, ALCANTARILLADO Y ASE...",Contribuciones,ACUEDUCTO,3.683291e+08


 ### ✅ **6. Verificación Final de Limpieza**

In [90]:
datasets = {
    "Consumo": df_consumo,
    "Empresas (Limpio)": df_empresas_limpio,
    "Reclamaciones Aseo": df_aseo,
    "Reclamaciones Energía": df_energia,
    "Reclamaciones Acueducto": df_acueducto,
    "Reclamaciones Alcantarillado": df_alcantarillado,
    "Reclamaciones Gas": df_gas
}

 ***Verificación General***


In [91]:
for nombre, df in datasets.items():
    print(f"\n🧾 Dataset: {nombre}")
    print("-" * 60)
    print(f"🔹 Dimensiones: {df.shape}")
    print(f"🔹 Columnas: {df.columns.tolist()}")
    print(f"🔹 Tipos:\n{df.dtypes}")
    print(f"🔹 Valores nulos:\n{df.isnull().sum()}")
    print(f"🔹 Duplicados: {df.duplicated().sum()}")
    display(df.head(3))



🧾 Dataset: Consumo
------------------------------------------------------------
🔹 Dimensiones: (306, 15)
🔹 Columnas: ['fecha', 'barrio', 'estrato', 'consumo_energia_kwh', 'consumo_agua_m3', 'consumo_gas_m3', 'reclamaciones', 'interrupciones', 'duracion_promedio_interrupciones', 'estado_medidor', 'tecnico_asignado', 'consumo_sospechoso', 'fraude_detectado', 'lectura_estimada', 'BARRIO_LIMPIO']
🔹 Tipos:
fecha                                object
barrio                               object
estrato                               int64
consumo_energia_kwh                 float64
consumo_agua_m3                     float64
consumo_gas_m3                      float64
reclamaciones                         int64
interrupciones                        int64
duracion_promedio_interrupciones    float64
estado_medidor                       object
tecnico_asignado                     object
consumo_sospechoso                   object
fraude_detectado                      int64
lectura_estimada      

,fecha,barrio,estrato,consumo_energia_kwh,consumo_agua_m3,consumo_gas_m3,reclamaciones,interrupciones,duracion_promedio_interrupciones,estado_medidor,tecnico_asignado,consumo_sospechoso,fraude_detectado,lectura_estimada,BARRIO_LIMPIO
0,2023-01-01,La SierrA,1,219.737132,18.617357,19.533820,1,0,50.230299,NaN,NaN,NaN,0,no,LA SIERRA
1,2023-01-01,la sierra,5,202.323303,30.105153,10.933853,1,1,29.748302,bien,Laura Pérez,NaN,0,sí,LA SIERRA
2,2023-01-01,La siera,3,245.315606,4.761240,12.003678,2,0,27.575932,bien,Carlos Ruiz,NaN,0,no,LA SIERRA



🧾 Dataset: Empresas (Limpio)
------------------------------------------------------------
🔹 Dimensiones: (3596, 5)
🔹 Columnas: ['MUNICIPIO', 'EMPRESA', 'MEDIDA', 'SERVICIO', 'VALOR']
🔹 Tipos:
MUNICIPIO     object
EMPRESA       object
MEDIDA        object
SERVICIO      object
VALOR        float64
dtype: object
🔹 Valores nulos:
MUNICIPIO    0
EMPRESA      0
MEDIDA       0
SERVICIO     0
VALOR        0
dtype: int64
🔹 Duplicados: 0


,MUNICIPIO,EMPRESA,MEDIDA,SERVICIO,VALOR
0,BARANOA,"82-SOCIEDAD DE ACUEDUCTO, ALCANTARILLADO Y ASE...",Facturación total,ACUEDUCTO,6.125393e+10
1,BARANOA,"82-SOCIEDAD DE ACUEDUCTO, ALCANTARILLADO Y ASE...",Consumo total,ACUEDUCTO,3.850631e+06
2,BARANOA,"82-SOCIEDAD DE ACUEDUCTO, ALCANTARILLADO Y ASE...",Numero de suscriptores,ACUEDUCTO,1.189200e+04



🧾 Dataset: Reclamaciones Aseo
------------------------------------------------------------
🔹 Dimensiones: (3670, 6)
🔹 Columnas: ['MUNICIPIO', 'TIPO DE TRÁMITE', 'DETALLE', 'Año', 'Número de Reclamos', 'Fecha']
🔹 Tipos:
MUNICIPIO                     object
TIPO DE TRÁMITE               object
DETALLE                       object
Año                            int64
Número de Reclamos             int64
Fecha                 datetime64[ns]
dtype: object
🔹 Valores nulos:
MUNICIPIO             0
TIPO DE TRÁMITE       0
DETALLE               0
Año                   0
Número de Reclamos    0
Fecha                 0
dtype: int64
🔹 Duplicados: 0


...

,MUNICIPIO,TIPO DE TRÁMITE,DETALLE,Año,Número de Reclamos,Fecha
0,BARANOA,QUEJA,SUSPENSION O CORTE DEL SERVICIO,2021,8,2021-01-01
1,BARANOA,QUEJA,INTERRUPCIONES EN LA PRESTACION DEL SERVICIO,2021,612,2021-01-01
2,BARANOA,QUEJA,ESTADO DE LAS INFRAESTRUCTURA,2021,3,2021-01-01



🧾 Dataset: Reclamaciones Alcantarillado
------------------------------------------------------------
🔹 Dimensiones: (3900, 6)
🔹 Columnas: ['MUNICIPIO', 'TIPO DE TRÁMITE', 'DETALLE', 'Año', 'Número de Reclamos', 'Fecha']
🔹 Tipos:
MUNICIPIO                     object
TIPO DE TRÁMITE               object
DETALLE                       object
Año                            int64
Número de Reclamos             int64
Fecha                 datetime64[ns]
dtype: object
🔹 Valores nulos:
MUNICIPIO             0
TIPO DE TRÁMITE       0
DETALLE               0
Año                   0
Número de Reclamos    0
Fecha                 0
dtype: int64
🔹 Duplicados: 0


,MUNICIPIO,TIPO DE TRÁMITE,DETALLE,Año,Número de Reclamos,Fecha
0,BARANOA,QUEJA,ESTADO DE LAS INFRAESTRUCTURA,2021,2,2021-01-01
1,BARANOA,QUEJA,SUSPENSION O CORTE DEL SERVICIO,2021,0,2021-01-01
2,BARANOA,QUEJA,INCONFORMIDAD CON EL CAMBIO O CON EL COBRO DEL...,2021,0,2021-01-01



🧾 Dataset: Reclamaciones Gas
------------------------------------------------------------
🔹 Dimensiones: (3075, 6)
🔹 Columnas: ['MUNICIPIO', 'TIPO DE TRÁMITE', 'DETALLE', 'Año', 'Número de Reclamos', 'Fecha']
🔹 Tipos:
MUNICIPIO                     object
TIPO DE TRÁMITE               object
DETALLE                       object
Año                            int64
Número de Reclamos             int64
Fecha                 datetime64[ns]
dtype: object
🔹 Valores nulos:
MUNICIPIO             0
TIPO DE TRÁMITE       0
DETALLE               0
Año                   0
Número de Reclamos    0
Fecha                 0
dtype: int64
🔹 Duplicados: 0


,MUNICIPIO,TIPO DE TRÁMITE,DETALLE,Año,Número de Reclamos,Fecha
0,BARANOA,QUEJA,ENTREGA INOPORTUNA O NO ENTREGA DE LA FACTURA,2021,19,2021-01-01
1,BARANOA,QUEJA,SUSPENSION O CORTE DEL SERVICIO,2021,10,2021-01-01
2,BARANOA,QUEJA,NO CONEXION DEL SERVICIO,2021,0,2021-01-01


###💾 **7.  Guardar Archivos Limpios**



Asegurarse de tener definida esta ruta en tu notebook:

In [108]:
# Ruta base del proyecto
base_path = '/content/drive/MyDrive/00_Data_ISES'
clean_path = os.path.join(base_path, '01_data_clean')

# Crear carpeta si no existe
os.makedirs(clean_path, exist_ok=True)


Guardar cada archivo como .csv

In [110]:
# Guardar DataFrame de consumo simulado
df_consumo.to_csv(os.path.join(clean_path, 'consumo_simulado_limpio.csv'), index=False)

# Guardar empresas reformateado
df_empresas_limpio.to_csv(os.path.join(clean_path, 'caracterizacion_empresas_limpio.csv'), index=False)

# Guardar reclamaciones por servicio
df_aseo.to_csv(os.path.join(clean_path, 'reclamaciones_aseo_limpio.csv'), index=False)
df_energia.to_csv(os.path.join(clean_path, 'reclamaciones_energia_limpio.csv'), index=False)
df_acueducto.to_csv(os.path.join(clean_path, 'reclamaciones_acueducto_limpio.csv'), index=False)
df_alcantarillado.to_csv(os.path.join(clean_path, 'reclamaciones_alcantarillado_limpio.csv'), index=False)
df_gas.to_csv(os.path.join(clean_path, 'reclamaciones_gas_limpio.csv'), index=False)

print("✅ Todos los archivos limpios han sido exportados a 01_data_clean/")


✅ Todos los archivos limpios han sido exportados a 01_data_clean/


---

## 📁 Archivos exportados limpios (carpeta `01_data_clean/`)

- `consumo_simulado_limpio.csv`
- `caracterizacion_empresas_limpio.csv`
- `reclamaciones_aseo_limpio.csv`
- `reclamaciones_energia_limpio.csv`
- `reclamaciones_acueducto_limpio.csv`
- `reclamaciones_alcantarillado_limpio.csv`
- `reclamaciones_gas_limpio.csv`